In [1]:
import pandas as pd
import numpy as np
import time
import json
from pprint import pprint

In [2]:
def getTagsDict(df):
    tag_dict = {}
    for index, row in df.iterrows():
        for tag in row['hashtags']:
            if tag in tag_dict:
                tag_dict[tag] = tag_dict[tag] + 1
            else:
                tag_dict[tag] = 1
    sort_dict = sorted(tag_dict.items(), key=lambda kv: kv[1])
    return sort_dict

def get_DataFrame_for_affinity(df,taglist):
    result_df = pd.DataFrame(0, index = np.arange(0,len(df)),columns=taglist)    
    for i in range(len(df)):
        list_of_row = list(df.iloc[i])[0]
        for tag in list_of_row:
            if tag in taglist:
                result_df[tag][i] = 1
    return result_df

In [3]:
## JSON Rawdata 파일 읽어오기
filename = 'allseason_20k_rawdata.json'
allseason = pd.read_json(filename)
base_df = allseason[['id','username','hashtags']]

## id 와 hashtag 만 남기기
base_df.index = base_df['id']
base_df = base_df.drop(columns=['username'])
base_df = base_df.drop(columns=['id'])

## 비어있는 해시태그 리스트 제거 / 중복되는 태그 제거
fashion_df = base_df[base_df.astype(str)['hashtags'] != '[]']
tag_df = list(fashion_df['hashtags'])
for i in range(len(tag_df)):
    tag_df[i] = list(set(tag_df[i]))
fashion_df['hashtags'] = tag_df

## n 개의 고빈도 태그만 사용하도록 결정
n_tags = 100
fashion_dict = getTagsDict(fashion_df)
fashion_dict_top = fashion_dict[-n_tags:]
fashion_top = []
for i in fashion_dict_top:
    fashion_top.append(i[0])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [4]:
## 해당 태그로 post-hashtag 매트릭스의 생성
start_time = time.time()
post_hashtag = get_DataFrame_for_affinity(fashion_df,fashion_top)
print("--- %s seconds ---" % (time.time() - start_time))

--- 22.125407218933105 seconds ---


In [6]:
## post corr 연산
start_time = time.time()
hashtag_post = post_hashtag.transpose()
post_corr = hashtag_post.corr()
print("--- %s seconds ---" % (time.time() - start_time))
post_corr.head()

--- 57.38438677787781 seconds ---


,0,1,2,3,4,5,6,7,8,9,...,17634,17635,17636,17637,17638,17639,17640,17641,17642,17643
0,1.000000,0.058621,0.449247,0.812320,0.354371,0.812320,0.263251,0.411260,2.286260e-01,0.812320,...,-0.040346,-0.073877,-0.044431,-0.061827,-0.073877,-0.017675,-0.025123,-0.030928,-0.035898,0.354371
1,0.058621,1.000000,0.084215,0.107143,0.279543,0.107143,0.025516,0.156772,1.910270e-17,0.107143,...,0.229416,0.210042,-0.126323,-0.015980,0.210042,-0.050252,-0.071429,0.058621,-0.102062,0.192186
2,0.449247,0.084215,1.000000,0.565445,0.509090,0.565445,0.163308,0.590817,3.284449e-01,0.565445,...,0.328445,0.247642,0.113475,0.045756,0.129717,-0.025392,-0.036092,0.202408,-0.051571,0.656226
3,0.812320,0.107143,0.565445,1.000000,0.454257,1.000000,0.335347,0.520708,2.949631e-01,1.000000,...,-0.032774,-0.060012,-0.036092,-0.050223,-0.060012,-0.014358,-0.020408,-0.025123,-0.029161,0.454257
4,0.354371,0.279543,0.509090,0.454257,1.000000,0.454257,0.114123,0.461527,4.088382e-01,0.454257,...,0.408838,0.161468,0.067682,-0.110561,0.063609,-0.031607,-0.044926,0.149532,-0.064194,0.389499


In [9]:
post_id = fashion_df.index.values
post_corr['post_id'] = pd.DataFrame(post_id)
post_corr.head()

,0,1,2,3,4,5,6,7,8,9,...,17635,17636,17637,17638,17639,17640,17641,17642,17643,post_id
0,1.000000,0.058621,0.449247,0.812320,0.354371,0.812320,0.263251,0.411260,2.286260e-01,0.812320,...,-0.073877,-0.044431,-0.061827,-0.073877,-0.017675,-0.025123,-0.030928,-0.035898,0.354371,BtOm-dkHn5v
1,0.058621,1.000000,0.084215,0.107143,0.279543,0.107143,0.025516,0.156772,1.910270e-17,0.107143,...,0.210042,-0.126323,-0.015980,0.210042,-0.050252,-0.071429,0.058621,-0.102062,0.192186,BtQp-IfHkzq
2,0.449247,0.084215,1.000000,0.565445,0.509090,0.565445,0.163308,0.590817,3.284449e-01,0.565445,...,0.247642,0.113475,0.045756,0.129717,-0.025392,-0.036092,0.202408,-0.051571,0.656226,BtRdH3UBQ6i
3,0.812320,0.107143,0.565445,1.000000,0.454257,1.000000,0.335347,0.520708,2.949631e-01,1.000000,...,-0.060012,-0.036092,-0.050223,-0.060012,-0.014358,-0.020408,-0.025123,-0.029161,0.454257,BtOpNQpApMI
4,0.354371,0.279543,0.509090,0.454257,1.000000,0.454257,0.114123,0.461527,4.088382e-01,0.454257,...,0.161468,0.067682,-0.110561,0.063609,-0.031607,-0.044926,0.149532,-0.064194,0.389499,BtRHyjZAem6


In [14]:
post_corr = post_corr.drop(['post_id'], axis=1)
post_corr.head()

,0,1,2,3,4,5,6,7,8,9,...,17634,17635,17636,17637,17638,17639,17640,17641,17642,17643
0,1.000000,0.058621,0.449247,0.812320,0.354371,0.812320,0.263251,0.411260,2.286260e-01,0.812320,...,-0.040346,-0.073877,-0.044431,-0.061827,-0.073877,-0.017675,-0.025123,-0.030928,-0.035898,0.354371
1,0.058621,1.000000,0.084215,0.107143,0.279543,0.107143,0.025516,0.156772,1.910270e-17,0.107143,...,0.229416,0.210042,-0.126323,-0.015980,0.210042,-0.050252,-0.071429,0.058621,-0.102062,0.192186
2,0.449247,0.084215,1.000000,0.565445,0.509090,0.565445,0.163308,0.590817,3.284449e-01,0.565445,...,0.328445,0.247642,0.113475,0.045756,0.129717,-0.025392,-0.036092,0.202408,-0.051571,0.656226
3,0.812320,0.107143,0.565445,1.000000,0.454257,1.000000,0.335347,0.520708,2.949631e-01,1.000000,...,-0.032774,-0.060012,-0.036092,-0.050223,-0.060012,-0.014358,-0.020408,-0.025123,-0.029161,0.454257
4,0.354371,0.279543,0.509090,0.454257,1.000000,0.454257,0.114123,0.461527,4.088382e-01,0.454257,...,0.408838,0.161468,0.067682,-0.110561,0.063609,-0.031607,-0.044926,0.149532,-0.064194,0.389499


In [15]:
without_minus = post_corr.applymap(lambda x: 0 if x < 0 else x)

In [16]:
without_minus

,0,1,2,3,4,5,6,7,8,9,...,17634,17635,17636,17637,17638,17639,17640,17641,17642,17643
0,1.000000,5.862104e-02,0.449247,0.812320,0.354371,0.812320,0.263251,0.411260,2.286260e-01,0.812320,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.354371
1,0.058621,1.000000e+00,0.084215,0.107143,0.279543,0.107143,0.025516,0.156772,1.910270e-17,0.107143,...,0.229416,0.210042,0.000000,0.000000,0.210042,0.000000,0.000000,0.058621,0.000000,0.192186
2,0.449247,8.421519e-02,1.000000,0.565445,0.509090,0.565445,0.163308,0.590817,3.284449e-01,0.565445,...,0.328445,0.247642,0.113475,0.045756,0.129717,0.000000,0.000000,0.202408,0.000000,0.656226
3,0.812320,1.071429e-01,0.565445,1.000000,0.454257,1.000000,0.335347,0.520708,2.949631e-01,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.454257
4,0.354371,2.795426e-01,0.509090,0.454257,1.000000,0.454257,0.114123,0.461527,4.088382e-01,0.454257,...,0.408838,0.161468,0.067682,0.000000,0.063609,0.000000,0.000000,0.149532,0.000000,0.389499
5,0.812320,1.071429e-01,0.565445,1.000000,0.454257,1.000000,0.335347,0.520708,2.949631e-01,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.454257
6,0.263251,2.551552e-02,0.163308,0.335347,0.114123,0.335347,1.000000,0.144005,1.873172e-01,0.335347,...,0.000000,0.000000,0.000000,0.000000,0.200082,0.000000,0.000000,0.000000,0.000000,0.114123
7,0.411260,1.567724e-01,0.590817,0.520708,0.461527,0.520708,0.144005,1.000000,2.967199e-01,0.520708,...,0.296720,0.104274,0.095719,0.028810,0.104274,0.000000,0.000000,0.181506,0.000000,0.461527
8,0.228626,1.910270e-17,0.328445,0.294963,0.408838,0.294963,0.187317,0.296720,1.000000e+00,0.294963,...,0.157895,0.160623,0.135242,0.000000,0.032125,0.000000,0.000000,0.228626,0.000000,0.248509
9,0.812320,1.071429e-01,0.565445,1.000000,0.454257,1.000000,0.335347,0.520708,2.949631e-01,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.454257


In [17]:
start_time = time.time()
sum_mat = without_minus.sum(axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.683680057525635 seconds ---


In [18]:
sum_mat.head()

0    1995.058330
1    1439.544715
2    3253.899973
3    2431.319099
4    2409.043277
dtype: float64

In [20]:
post_id = fashion_df.index.values
ids = pd.DataFrame(post_id)

In [21]:
corr_score = pd.DataFrame(sum_mat, columns = ['score'])
corr_score['id'] = ids
corr_score.head()

,score,id
0,1995.058330,BtOm-dkHn5v
1,1439.544715,BtQp-IfHkzq
2,3253.899973,BtRdH3UBQ6i
3,2431.319099,BtOpNQpApMI
4,2409.043277,BtRHyjZAem6


In [22]:
corr_score.sort_values(by = ['score'], ascending = False)

,score,id
9109,4672.575454,BsspPOZhm9b
13190,4427.488125,BtNQH9LHxO9
4483,4427.488125,BtNQH9LHxO9
6619,4418.775094,BtOvWYgHzNz
5952,4418.775094,BtLCm29nM6G
11914,4418.775094,BtDrDKCHDjt
4709,4418.775094,BtLDckbnUCs
17592,4418.775094,BtK_FD3HK0V
7085,4418.775094,BtL0x9iFafp
7498,4418.775094,BtAkLHknXVy
